# MF

In [20]:
# 加载依赖
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from utils.movielens_dataset import MovieLensDatasetWithTorch
from utils.metric import Metric
from utils.trainer import Trainer

In [21]:
# 使用的超参数
config = {
    'TRAIN_BATCH_SIZE': 8,
    'VALID_BATCH_SIZE': 4,
    'TEST_BATCH_SIZE': 4,
    'DATASET_RATIO': [0.8, 0.1, 0.1],
    'DEVICE': 'cpu',
    'NUM_WORKERS': 0,
    'EPOCH': 30,
    'NUM_FEATURE': 111,
    'POS_WEIGHT': 2,
    'LEARNING_RATE': 1e-3,
    'K': 8,
    'TASK': 'recommend',
    'USER_NUM': 611,
    'ITEM_NUM': 193610,
}

In [22]:
# 加载数据集
# MovieLens
with open('../dataset/ml-latest-small-ratings.txt', 'r', encoding='utf-8') as f:
    ml_dataset = MovieLensDatasetWithTorch(f, task='rating')
dataset_size = len(ml_dataset)
dataset_split_size = [int(dataset_size * r) for r in config['DATASET_RATIO']]
if sum(dataset_split_size) != dataset_size:
    dataset_split_size[-1] += dataset_size - sum(dataset_split_size)
train_set, valid_set, test_set = torch.utils.data.random_split(ml_dataset, dataset_split_size)

user_num, item_num = ml_dataset.user_num, ml_dataset.item_num

In [23]:
ml_dataset[610, 193609]

[(610, 193609), tensor(0.)]

In [24]:
train_loader = DataLoader(
    dataset=train_set,
    batch_size=config['TRAIN_BATCH_SIZE'],
    shuffle=True,
    num_workers=config['NUM_WORKERS']
)

valid_loader = DataLoader(
    dataset=valid_set,
    batch_size=config['VALID_BATCH_SIZE'],
    shuffle=False,
    num_workers=config['NUM_WORKERS']
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=config['TEST_BATCH_SIZE'],
    shuffle=False,
    num_workers=config['NUM_WORKERS']
)

In [25]:
# 构建模型
class MF(nn.Module):
    def __init__(self, user_num, item_num, k):
        super(MF, self).__init__()
        U = torch.zeros((user_num, k))
        I = torch.zeros((item_num, k))
        user_bias = torch.zeros((user_num, k))
        item_bias = torch.zeros((item_num, k))

        nn.init.xavier_uniform_(U)
        nn.init.xavier_uniform_(I)
        nn.init.xavier_uniform_(user_bias)
        nn.init.xavier_uniform_(item_bias)

        self.U = nn.Parameter(U, requires_grad=True)
        self.I = nn.Parameter(I, requires_grad=True)
        self.user_bias = nn.Parameter(user_bias, requires_grad=True)
        self.item_bias = nn.Parameter(item_bias, requires_grad=True)

    def forward(self, batch_data):
        user_idx_lst = batch_data[0]
        sparse_indices = batch_data[1].coalesce().indices()
        for i in range(len(user_idx_lst)):
            sparse_indices[0, :] = torch.where(sparse_indices[0, :] == i, user_idx_lst[i], sparse_indices[0, :])
        user_idx = sparse_indices[0, :]
        item_idx = sparse_indices[1, :]

        user_factor = self.U[user_idx, :]
        item_factor = self.I[item_idx, :]
        user_bias = self.user_bias[user_idx, :]
        item_bias = self.item_bias[item_idx, :]
        # print(user_factor.shape)
        # print(item_factor.shape)
        return torch.sum(user_factor * item_factor + user_bias + item_bias, dim=1)

    def rec(self, user_idx_lst, k=50):
        # print(user_idx_lst)
        return torch.topk(torch.matmul(self.U[user_idx_lst, :], self.I.t()), k=k, dim=1)

In [26]:
class MFTrainer(Trainer):
    def step(self, batch_data, mode='train', **param_for_rec):
        device = self.config['DEVICE']

        def compute_pred(batch_data):
            # print(batch_data[1].shape)
            pred = self.model(batch_data)
            loss = self.loss_func(pred, batch_data[1].coalesce().values())
            gt_rec_lst = param_for_rec['gt_rec_lst']
            pred_rec_lst = param_for_rec['pred_rec_lst']
            if mode == 'evaluate':
                user_idx_lst = batch_data[0]
                sparse_indices = batch_data[1].coalesce().indices()
                for i in range(len(user_idx_lst)):
                    tmp_idx = torch.nonzero(sparse_indices[0, :] == i).view(-1)
                    gt_rec_lst.extend(sparse_indices[1, tmp_idx].tolist())
                pred_rec_lst.extend((self.model.rec(user_idx_lst)[1] + 1).tolist())
                self.metric.compute_metric(gt_rec_lst, pred_rec_lst, 200000, len(gt_rec_lst))

            return pred, loss, gt_rec_lst, pred_rec_lst

        if mode == 'train':
            self.model.train()
            self.optimizer.zero_grad()
            pred, loss = compute_pred(batch_data)
            loss.backward()
            self.optimizer.step()
            return loss.item(), pred
        elif mode == 'evaluate':
            with torch.no_grad():
                self.model.eval()
                pred, loss, gt_rec_lst, pred_rec_lst = compute_pred(batch_data)

                return loss.item(), pred, gt_rec_lst, pred_rec_lst
        else:
            raise ValueError("Wrong Mode")

In [27]:
model = MF(user_num, item_num, k=config['K'])
optimizer = optim.Adam(lr=config['LEARNING_RATE'], params=model.parameters())
loss_func = nn.MSELoss()
metric = Metric(k=(1, 10, 50))

trainer = Trainer(
    model=model,
    loss_func=loss_func,
    optimizer=optimizer,
    metric=metric,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    config=config,
)

==========Config==========
TRAIN_BATCH_SIZE: 8
VALID_BATCH_SIZE: 4
TEST_BATCH_SIZE: 4
DATASET_RATIO: [0.8, 0.1, 0.1]
DEVICE: cpu
NUM_WORKERS: 0
EPOCH: 30
NUM_FEATURE: 111
POS_WEIGHT: 2
LEARNING_RATE: 0.001
K: 8
TASK: recommend
USER_NUM: 611
ITEM_NUM: 193610


In [28]:
if __name__ == '__main__':
    trainer.train()
    trainer.test()

==========TRAIN BEGIN==========


100%|██████████| 61/61 [00:00<00:00, 114.16it/s]


Train Epoch: 1
Loss: 12.914455585792416


100%|██████████| 16/16 [00:00<00:00, 319.88it/s]


Valid Epoch: 1
hit:
	hit@1: 0.0492
	hit@10: 0.2623
	hit@50: 0.5574
gini_index:
	gini_index@1: 0.1157
	gini_index@10: 0.2734
	gini_index@50: 0.3429
diversity:
	diversity@1: 0.9951
	diversity@10: 0.9785
	diversity@50: 0.9656
nDCG:
	nDCG@1: 0.0492
	nDCG@10: 0.1418
	nDCG@50: 0.1983
MRR:
	MRR@1: 0.0492
	MRR@10: 0.1076
	MRR@50: 0.1211



...

Train Epoch: 30
Loss: 0.7711418886653713


100%|██████████| 16/16 [00:00<00:00, 260.20it/s]


Valid Epoch: 30
hit:
	hit@1: 0.0328
	hit@10: 0.2131
	hit@50: 0.5082
gini_index:
	gini_index@1: 0.2208
	gini_index@10: 0.3166
	gini_index@50: 0.3709
diversity:
	diversity@1: 0.9858
	diversity@10: 0.9712
	diversity@50: 0.9549
nDCG:
	nDCG@1: 0.0328
	nDCG@10: 0.1144
	nDCG@50: 0.1892
MRR:
	MRR@1: 0.0328
	MRR@10: 0.0846
	MRR@50: 0.0997

==========TRAIN END==========


100%|██████████| 16/16 [00:00<00:00, 250.61it/s]


Test Loss: 2.5238191708922386
hit:
	hit@1: 0.1290
	hit@10: 0.2581
	hit@50: 0.3387
gini_index:
	gini_index@1: 0.2309
	gini_index@10: 0.3276
	gini_index@50: 0.3660
diversity:
	diversity@1: 0.9857
	diversity@10: 0.9710
	diversity@50: 0.9562
nDCG:
	nDCG@1: 0.1290
	nDCG@10: 0.1771
	nDCG@50: 0.1775
MRR:
	MRR@1: 0.1290
	MRR@10: 0.1645
	MRR@50: 0.1677



In [29]:
model.rec([0, 1])

torch.return_types.topk(
values=tensor([[0.1160, 0.1099, 0.1079, 0.1066, 0.1062, 0.1055, 0.1046, 0.1045, 0.1043,
         0.1030, 0.1030, 0.1023, 0.1022, 0.1005, 0.1001, 0.0999, 0.0994, 0.0986,
         0.0984, 0.0980, 0.0977, 0.0975, 0.0968, 0.0967, 0.0964, 0.0962, 0.0962,
         0.0959, 0.0957, 0.0956, 0.0956, 0.0955, 0.0953, 0.0952, 0.0952, 0.0949,
         0.0949, 0.0943, 0.0942, 0.0942, 0.0941, 0.0941, 0.0940, 0.0939, 0.0938,
         0.0936, 0.0933, 0.0933, 0.0933, 0.0932],
        [0.7879, 0.7620, 0.7618, 0.7144, 0.7096, 0.7091, 0.7006, 0.6990, 0.6962,
         0.6919, 0.6863, 0.6770, 0.6751, 0.6740, 0.6729, 0.6695, 0.6673, 0.6663,
         0.6657, 0.6643, 0.6625, 0.6603, 0.6598, 0.6594, 0.6591, 0.6589, 0.6585,
         0.6582, 0.6571, 0.6542, 0.6537, 0.6501, 0.6492, 0.6477, 0.6440, 0.6439,
         0.6430, 0.6429, 0.6428, 0.6424, 0.6418, 0.6416, 0.6411, 0.6397, 0.6393,
         0.6390, 0.6385, 0.6381, 0.6380, 0.6374]], grad_fn=<TopkBackward0>),
indices=tensor([[  1394,   1272